# Semi-supervised Dataset Preparation

- Mục tiêu: tạo bộ dữ liệu giữ cả phần **chưa có nhãn AQI** (aqi_class = NaN) để dùng cho self-training/co-training.
- Đồng thời **giả lập thiếu nhãn trong TRAIN** (time-aware) để mini project có thể thử nhiều mức thiếu nhãn.

In [1]:
CLEANED_PATH = "data/processed/cleaned.parquet"
OUTPUT_SEMI_DATASET_PATH = "data/processed/dataset_for_semi.parquet"
CUTOFF = "2017-01-01"
LABEL_MISSING_FRACTION = 0.95
RANDOM_STATE = 42

In [6]:
from pathlib import Path
import pandas as pd
import sys
import os
from pathlib import Path

# 1. Xác định vị trí của notebook hiện tại
current_dir = Path('.').resolve()

# 2. Nếu đang ở trong thư mục 'notebooks', hãy lùi ra thư mục gốc 'air_guard'
if current_dir.name == 'notebooks':
    PROJECT_ROOT = current_dir.parent
else:
    PROJECT_ROOT = current_dir

# 3. Tạo đường dẫn đầy đủ đến file cleaned.parquet
# Giả sử CLEANED_PATH của bạn là "data/processed/cleaned.parquet"
full_cleaned_path = (PROJECT_ROOT / "data" / "processed" / "cleaned.parquet").resolve()

print(f"Đang kiểm tra file tại: {full_cleaned_path}")

# 4. Kiểm tra sự tồn tại của file trước khi đọc
if full_cleaned_path.exists():
    df = pd.read_parquet(full_cleaned_path)
    print("Đọc dữ liệu thành công!")
else:
    print(f"LỖI: Không tìm thấy file tại {full_cleaned_path}")
    print("Vui lòng kiểm tra xem bạn đã chạy bước tạo file cleaned.parquet chưa.")
from src.semi_supervised_library import SemiDataConfig, mask_labels_time_aware

PROJECT_ROOT = Path(".").resolve()
if not (PROJECT_ROOT / "data").exists() and (PROJECT_ROOT.parent / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent.resolve()


cfg = SemiDataConfig(cutoff=CUTOFF, random_state=int(RANDOM_STATE))
df2 = mask_labels_time_aware(df, cfg, missing_fraction=float(LABEL_MISSING_FRACTION))

out_path = (PROJECT_ROOT / OUTPUT_SEMI_DATASET_PATH).resolve()
out_path.parent.mkdir(parents=True, exist_ok=True)
df2.to_parquet(out_path, index=False)

print("Saved:", out_path)
print("Rows:", len(df2))
print("Labeled ratio:", float(df2["is_labeled"].mean()) if "is_labeled" in df2.columns else None)

Đang kiểm tra file tại: D:\3\Data_Mining\Mini_Prj\air_guard\data\processed\cleaned.parquet
Đọc dữ liệu thành công!
Saved: D:\3\Data_Mining\Mini_Prj\air_guard\notebooks\data\processed\dataset_for_semi.parquet
Rows: 420768
Labeled ratio: 0.08671049129211347
